# Notes, 7/9/19

## Initial attempt to compile

* ODC requires eckit 1.1.0
* ecbuild version doesn't matter
* "Fortran" missing from project statement in top-level CMakeLists.txt file
* No python API
* Got it to compile and all tests pass
    * Tests download data - first run through ctest takes 40 minutes!
    * Subsequent testing goes fast since don't have to download test data

# Notes, 7/15/19

* Reader first
* Radiosonde, AMSU-A

* Mark O. upgrade to eckit 1.1.0
    * PR's are ready

* Simon Smart is contact for ODC

# Notes, 7/15/19

* Python interface is called odyssey
    * Simon sent a tarball

* odc CMake config produces
    * libodccore
    * libodctest
    * libodctools
    
* odyssey uses libodccore

# Notes, 7/19/19

* Created odc-bundle
    * odc
    * odyssey

## python

* odyssey uses python setup.py

* use the setuptools pkg
    * packaging system built on older distutils pkg

* to save output from setup.py
~~~~~~~~
python setup.py install --record <log_file>
~~~~~~~~

* use the wheel file format for distribution
    * egg is an older format that will eventually get depricated
    * PiPY site is well uder way migrating from egg to wheel
    * to create and install a wheel file for odyessey
~~~~~~~~
# cd to the ecbuild build directory:

python setup bdist_wheel        # create the wheel file

cd dist                         # Important to do this for the following pip
                                # to work properly. Without this, the pip command
                                # (for some reason) simply points the install area
                                # to the files in the build area instead of placing
                                # copies in the install area

pip uninstall odyssey
pip install <wheel_file>
~~~~~~~~

* handy web sites
    * packaging.python.org/
    * setuptools.readthedocs.io/
    * python101.pythonlibrary.org/
    * python.packaging.readthedocs.io/en/latest/index.html


# Notes, 7/25/19

* had to write a script to do the build/install using setup.py
     * Difficult to locate wheel file
         * wheel file name changes based on platform, os, etc.
         * difficult to predit the name of the wheel file
         * difficult to use file globbing in CMake scripts
     * odyssey_setup.sh action dist_file install_dir
         * action is: build, install
         * dist_file: text file that holds the path to the wheel file
            * if action is build, then dist_file is created
            * if action is install, then dist_file is read


# Notes, 7/30/19

## Python installations

* odyssey install works with miniconda2, 3
* Does not work with container python2, 3
    * wheel file gets built okay, but pip does not install it
* pip doesn't contain the "pep425tags"
    * already upgrading pip
    * not upgrading wheel (could be this)
    * pip is found in /usr/local/bin, but python2,3 are installed in /usr/bin
* Worked with Mark and discovered that the recommended way to run pip is through the python command
~~~~~~~
python -m pip install ...
~~~~~~~

* Running pip through python gets the correct version of pip
    * Works in the containers

# Notes, 8/1/19

* Odyssey needs the following packages
    * pandas
    * cffi     (for loading libodccode)
    * pycparser
    * pytest

# Notes, 9/9/19

* AMSU-A, ATOVS example ODC file from Steve V
    * Cheyenne: /glade/work/vahl/MO_Data/OdbApi/atovs/0000/52005.odb
    * 38,333,328 rows
    * 270 columns
* To convert to netcdf
    * use codyssey to decode odb file into a pandas frame
    * convert pandas frame to xarray
    * use xarray to_netcdf method to create netcdf file
        * Encoding
        * Build a dictionary indexed by variable name to specify netcdf encodings
           * Can set things like \_FillValue, compression, etc.
           * Example:
               * xarray uses NaN as netcdf fill value
               * Remove nan as fill value
               * Enable compression
~~~~~~~~
{ 'variable1' : {
  '_FillValue' : None,
  'zlib' : True,
  'complev' : 6 } }
~~~~~~~~

* Python packages required for this:
    * pandas
    * xarray

* Performance testing
    * Start with large files (AMSU-A, ATOVS)
    * ODB and netcdf files with identical contents
    * Just read in the contents and time that piece
        * Any conversion/reformatting to get into IODA memory should be similar for radiance data
        * ODB has giant vector, but channels are regularly spaced so should be able to view as 2D array
            * Ie, the linear memory storage matches what is needed for 2D array

# Notes, 10/9/19

* The Fortran interface is the most developed one since the first client of ODC was using Fortran
* The C++ interface is just a place holder at this point and will be going away
* The C interface is nearly a one-for-one mapping with the Fortran interface
    * Fortran test code can be used to guide how to use the C interface

| Fortran routine | C routine |
|:----------------|:----------|
|odc_api_initialise|odc_initialise_api|
|||
|reader%open_path|odc_open_path|
|reader%close|odc_close|
|||
|frame%initialise|odc_new_frame|
|frame%next|odc_next_frame|
|frame%free|odc_free_frame|
|||
|decoder%initilise|odc_new_decoder|
|decoder%defaults_from_frame|odc_decoder_defaults_from_frame|
|decoder%decode|odc_decode|
|decoder%data|odc_decoder_data_array|
|decoder%free|odc_free_decoder|

* Use the Fortran code as a guide for development using the C inteface
    * Nearly one-to-one mapping of API routines
    * Use odc_ls.F90 as a guide instead of the "odc ls" command

## Frame layout

* The data array is an array of void\* in 8-byte chunks
* Each element (column) in the frame is 8 bytes of void\*
    * This is 8 characters for a string
    * To store longer strings, multiple columns are allocated
        * Eg, 20 byte timestamp would need 3 columns (3 * 8 = 24 bytes)
    * Column attributes
        * name
        * type (ODC_INTEGER, ODC_REAL, ODC_DOUBLE, ODC_STRING, ODC_BITFIELD)
        * element_size (number of bytes)
        * element_size_doubles (number of columns)
        * bitfield_count (number of bits that are of signficance in the column)
    * In order to parse the frame, you must keep track of the starting column and size (number of columns) for each item
* By default frames contain a maximum of 10000 rows
* Set ODC_INTEGERS_AS_LONGS, which will store the bit pattern for a long in the frame data array column
    * This will avoid the need to do the static_cast step
